In [3]:
!pip install transformers


     |████████████████████████████████| 1.5MB 16.9MB/s 
     |████████████████████████████████| 890kB 46.3MB/s 
     |████████████████████████████████| 2.9MB 50.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=a3f63c632c42b7831034a696d19580b879d63177dbe6173ba97db823034438a2
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [4]:
import torch

import random
import numpy as np

SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [5]:
from transformers import BertTokenizer

#loading tokenizer for bert model which ignores case
#tokenizer allows for words in a string to be split and tokenized for the bert/
#model to interperet 

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [6]:
#defining the special tokens that need to be added to the input text

init_token_idx = tokenizer.cls_token_id
eos_token_idx = tokenizer.sep_token_id
pad_token_idx = tokenizer.pad_token_id
unk_token_idx = tokenizer.unk_token_id

#BERT only works on samples that are less then or equal to the ones it was /
#trained on (512 tokens)
max_input_length = tokenizer.max_model_input_sizes['bert-base-uncased']

In [7]:
#define function to pass text to so it can be tokenized
def tokenize_and_cut(sentence):
    tokens = tokenizer.tokenize(sentence) 
    tokens = tokens[:max_input_length-2] # it is 510 in this case becauxe the max length for the/
  #model is 512 but we must add 2 tokens ([CLS] and [SEP])
    return tokens

In [8]:
from torchtext import data

# loading our text and labels in the proper manner
TEXT = data.Field(batch_first = True,
                  use_vocab = False,
                  tokenize = tokenize_and_cut,
                  preprocessing = tokenizer.convert_tokens_to_ids,
                  init_token = init_token_idx,
                  eos_token = eos_token_idx,
                  pad_token = pad_token_idx,
                  unk_token = unk_token_idx)

LABEL = data.LabelField(dtype = torch.float)

In [10]:
from torchtext import datasets

# getting our training, validation, and testing splits
train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

train_data, valid_data = train_data.split(random_state = random.seed(SEED))


aclImdb_v1.tar.gz:   0%|          | 0.00/84.1M [00:00<?, ?B/s]

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:03<00:00, 22.7MB/s]


In [11]:
#check the numbers of each
print(f"Number of training examples: {len(train_data)}")
print(f"Number of validation examples: {len(valid_data)}")
print(f"Number of testing examples: {len(test_data)}")


Number of training examples: 17500
Number of validation examples: 7500
Number of testing examples: 25000


In [12]:
#check an example from the training split
print(vars(train_data.examples[6]))

{'text': [1011, 1027, 3397, 27594, 2121, 2013, 2119, 2434, 1998, 3185, 4617, 1027, 1011, 1026, 7987, 1013, 1028, 1026, 7987, 1013, 1028, 1045, 2572, 1037, 4121, 5470, 1997, 10289, 8214, 2139, 3000, 3315, 1010, 1045, 2036, 3191, 2338, 2023, 2621, 1998, 1045, 3427, 3185, 2651, 1010, 1999, 1996, 2927, 1045, 2001, 17772, 2000, 3422, 1037, 2204, 2143, 2926, 2043, 1045, 2387, 1020, 1012, 1017, 10047, 18939, 2391, 1012, 2034, 5019, 2020, 2204, 2021, 1045, 2941, 4741, 2005, 1010, 12370, 3496, 3495, 2013, 2338, 1010, 2021, 2009, 2001, 3671, 2021, 2043, 9686, 28990, 2850, 8537, 5861, 3995, 7442, 1045, 2371, 2242, 2003, 3308, 1012, 1012, 2027, 2123, 1005, 1056, 2130, 3610, 1999, 2338, 1010, 2130, 1999, 3315, 1012, 1012, 1026, 7987, 1013, 1028, 1026, 7987, 1013, 1028, 1998, 2045, 2020, 4121, 6578, 2339, 9686, 28990, 2850, 2108, 13224, 2011, 23324, 1010, 1999, 1996, 2338, 2016, 12011, 2014, 13555, 2129, 2000, 6869, 3616, 1997, 5119, 2030, 4339, 2773, 1000, 6887, 8913, 8286, 1000, 1012, 1012, 3524, 

In [13]:
# the text from that example
tokens = tokenizer.convert_ids_to_tokens(vars(train_data.examples[6])['text'])

print(tokens)

['-', '=', 'contains', 'spoil', '##er', 'from', 'both', 'original', 'and', 'movie', 'versions', '=', '-', '<', 'br', '/', '>', '<', 'br', '/', '>', 'i', 'am', 'a', 'huge', 'fan', 'of', 'notre', 'dame', 'de', 'paris', 'musical', ',', 'i', 'also', 'read', 'book', 'this', 'summer', 'and', 'i', 'watched', 'movie', 'today', ',', 'in', 'the', 'beginning', 'i', 'was', 'hopeful', 'to', 'watch', 'a', 'good', 'film', 'especially', 'when', 'i', 'saw', '6', '.', '3', 'im', '##db', 'point', '.', 'first', 'scenes', 'were', 'good', 'but', 'i', 'actually', 'waited', 'for', ',', 'theaters', 'scene', 'directly', 'from', 'book', ',', 'but', 'it', 'was', 'normal', 'but', 'when', 'es', '##meral', '##da', 'kisses', 'grin', '##go', '##ire', 'i', 'felt', 'something', 'is', 'wrong', '.', '.', 'they', 'don', "'", 't', 'even', 'kiss', 'in', 'book', ',', 'even', 'in', 'musical', '.', '.', '<', 'br', '/', '>', '<', 'br', '/', '>', 'and', 'there', 'were', 'huge', 'gap', 'why', 'es', '##meral', '##da', 'being', 'jud

In [14]:
#build vocab for the labels
LABEL.build_vocab(train_data)

In [15]:
# check the label voacb (0 = negative, 1 = positive)
print(LABEL.vocab.stoi)

defaultdict(<function _default_unk_index at 0x7f44c878e730>, {'neg': 0, 'pos': 1})


In [16]:
BATCH_SIZE = 128

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, 
    device = device)

In [17]:
from transformers import BertTokenizer, BertModel

#getting the pretrained model

bert = BertModel.from_pretrained('bert-base-uncased')

In [18]:
import torch.nn as nn

class BERTGRUSentiment(nn.Module):
    def __init__(self,
                 bert,
                 hidden_dim,
                 output_dim,
                 n_layers,
                 bidirectional,
                 dropout):
        
        super().__init__()
        
        self.bert = bert
        
        embedding_dim = bert.config.to_dict()['hidden_size']
        
        self.rnn = nn.GRU(embedding_dim,
                          hidden_dim,
                          num_layers = n_layers,
                          bidirectional = bidirectional,
                          batch_first = True,
                          dropout = 0 if n_layers < 2 else dropout)
        
        self.out = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        
                
        with torch.no_grad():
            embedded = self.bert(text)[0]
                
        
        _, hidden = self.rnn(embedded)
        
        
        if self.rnn.bidirectional:
            hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        else:
            hidden = self.dropout(hidden[-1,:,:])
                
        
        output = self.out(hidden)
        
        
        return output

In [19]:
# create an instance of the model
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.25

model = BERTGRUSentiment(bert,
                         HIDDEN_DIM,
                         OUTPUT_DIM,
                         N_LAYERS,
                         BIDIRECTIONAL,
                         DROPOUT)

In [20]:
# freeze parameters from BERT so we don't train them
for name, param in model.named_parameters():                
    if name.startswith('bert'):
        param.requires_grad = False

In [21]:
# show the parameters we are training
for name, param in model.named_parameters():                
    if param.requires_grad:
        print(name)

rnn.weight_ih_l0
rnn.weight_hh_l0
rnn.bias_ih_l0
rnn.bias_hh_l0
rnn.weight_ih_l0_reverse
rnn.weight_hh_l0_reverse
rnn.bias_ih_l0_reverse
rnn.bias_hh_l0_reverse
rnn.weight_ih_l1
rnn.weight_hh_l1
rnn.bias_ih_l1
rnn.bias_hh_l1
rnn.weight_ih_l1_reverse
rnn.weight_hh_l1_reverse
rnn.bias_ih_l1_reverse
rnn.bias_hh_l1_reverse
out.weight
out.bias


In [22]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

In [23]:
criterion = nn.BCEWithLogitsLoss()

In [24]:
model = model.to(device)
criterion = criterion.to(device)

In [25]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [26]:
from tqdm import tqdm

def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    with tqdm(iterator, unit = "batch") as tepoch:
      for batch in tepoch:
        optimizer.zero_grad()
        
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [27]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
      with tqdm(iterator, unit= "batch") as tepoch:
        
        for batch in tepoch:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [28]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [29]:
N_EPOCHS = 5

#training the modelwith tqdm for progress bar

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
        
    end_time = time.time()
        
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
        
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut6-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

  0%|          | 0/137 [00:00<?, ?batch/s]

Epoch: 01 | Epoch Time: 17m 29s
	Train Loss: 0.487 | Train Acc: 75.54%
	 Val. Loss: 0.282 |  Val. Acc: 88.94%


  0%|          | 0/137 [00:00<?, ?batch/s]

Epoch: 02 | Epoch Time: 17m 46s
	Train Loss: 0.285 | Train Acc: 88.58%
	 Val. Loss: 0.222 |  Val. Acc: 91.29%


  0%|          | 0/137 [00:00<?, ?batch/s]

Epoch: 03 | Epoch Time: 17m 45s
	Train Loss: 0.236 | Train Acc: 90.42%
	 Val. Loss: 0.243 |  Val. Acc: 89.96%


  0%|          | 0/137 [00:00<?, ?batch/s]

Epoch: 04 | Epoch Time: 17m 46s
	Train Loss: 0.206 | Train Acc: 91.92%
	 Val. Loss: 0.215 |  Val. Acc: 91.90%


100%|██████████| 59/59 [02:56<00:00,  3.00s/batch]

Epoch: 05 | Epoch Time: 17m 44s
	Train Loss: 0.184 | Train Acc: 92.99%
	 Val. Loss: 0.223 |  Val. Acc: 91.40%


In [31]:
# load the best model and try it on some text

model.load_state_dict(torch.load("tut6-model.pt"))
test_loss, test_acc = evaluate(model, test_iterator, criterion)
print(f"Test Loss: {test_loss:.3f} | Test Acc: {test_acc:.2f}")

100%|██████████| 196/196 [09:30<00:00,  2.91s/batch]

Test Loss: 0.208 | Test Acc: 0.92


In [41]:
def predict_sentiment(model, tokenizer, sentence):
  model.eval()
  tokens = tokenizer.tokenize(sentence)
  tokens = tokens[:max_input_length-2]
  indexed = [init_token_idx] + tokenizer.convert_tokens_to_ids(tokens) + [eos_token_idx]
  tensor = torch.LongTensor(indexed).to(device)
  tensor = tensor.unsqueeze(0)
  prediction = torch.sigmoid(model(tensor))
  review_type = ""
  if prediction.item() > 0.6:
    review_type = "Positive Sentiment"
  elif prediction.item() < 0.4:
    review_type = "Negative sentiment"
  else:
    review_type = "Neutral sentiment"
  return (f"The sentiment is: {review_type} | Prediction : {prediction.item()}")

In [50]:
#Given a relatively ambiguous review
predict_sentiment(model, tokenizer, "I hate that I love this product. It is awfully good but terribly expensive")

'The sentiment is: Neutral sentiment | Prediction : 0.5239695310592651'

In [51]:
#Given a positive review
predict_sentiment(model, tokenizer, "WOW! Amazing device! I can't believe I didn't get it sooner!")

'The sentiment is: Positive Sentiment | Prediction : 0.8823106288909912'

In [53]:
# Given a positive review
predict_sentiment(model, tokenizer, "Such terrible acting and poor directing. Furthermore, the costumes didn't fit and the plot was non-existent.")

'The sentiment is: Negative sentiment | Prediction : 0.011832216754555702'